In [1]:
"""
Py-Microgrid Hybrid System Simulation Example
-----------------------------------
This example demonstrates how to:
1. Set up a hybrid system simulation
2. Download solar and wind resource data
3. Configure system parameters
4. Run optimization
5. Analyze and save results

Required files:
- Base YAML configuration file
- CSV file containing location data
"""

import os
import pandas as pd
from typing import Dict, List, Any

from hopp.utilities import ConfigManager
from hopp.utilities.keys import set_developer_nrel_gov_key
from hopp.tools.optimization import SystemOptimizer, LoadAnalyzer
from hopp.tools.analysis.bos import EconomicCalculator
from hopp.simulation.resource_files import ResourceDataManager

class HybridOptimizer:
    """Wrapper class for hybrid system optimization."""
    
    def __init__(self, 
                 yaml_file_path: str,
                 api_key: str,
                 email: str,
                 project_lifetime: int = 25,
                 discount_rate: float = 0.0588,
                 enable_flexible_load: bool = False,  # Default to no flexible load
                 max_load_reduction_percentage: float = 0.2):
        """
        Initialize hybrid system optimizer.
        
        Args:
            yaml_file_path: Path to YAML configuration file
            api_key: NREL API key for resource data
            email: Email for API authentication
            project_lifetime: Project lifetime in years
            discount_rate: Discount rate for economic calculations
            enable_flexible_load: Whether to enable flexible load management
            max_load_reduction_percentage: Maximum load reduction (if flexible load enabled)
        """
        self.yaml_file_path = yaml_file_path
        self.api_key = api_key
        self.email = email
        
        # Set up components
        self.resource_manager = ResourceDataManager(api_key, email)
        self.economic_calculator = EconomicCalculator(discount_rate, project_lifetime)
        self.system_optimizer = SystemOptimizer(
            yaml_file_path, 
            self.economic_calculator,
            enable_flexible_load=enable_flexible_load,
            max_load_reduction_percentage=max_load_reduction_percentage
        )
        
        # Define optimization bounds
        self.bounds = [
            (5000, 50000),    # PV capacity (kW)
            (1, 50),          # Wind turbines (1MW each)
            (5000, 30000),    # Battery energy capacity (kWh)
            (1000, 10000),    # Battery power capacity (kW)
            (17000, 30000)    # Genset capacity (kW)
        ]

    def process_location(self, latitude: float, longitude: float, location_id: str = "") -> Dict[str, Any]:
        """Process single location optimization."""
        print(f"\nProcessing location {location_id} at ({latitude}, {longitude})")
        
        try:
            # Download solar and wind resource data
            solar_path = self.resource_manager.download_solar_data(
                latitude, longitude, "2020"
            )
            wind_path = self.resource_manager.download_wind_data(
                latitude, longitude, "20200101", "20201231"
            )
            
            # Update configuration with location and resource data
            config = self.system_optimizer.config_manager.load_yaml_safely(self.yaml_file_path)
            config['site']['data']['lat'] = latitude
            config['site']['data']['lon'] = longitude
            config['site']['solar_resource_file'] = solar_path.replace('\\', '/')
            config['site']['wind_resource_file'] = wind_path.replace('\\', '/')
            self.system_optimizer.config_manager.save_yaml_safely(config, self.yaml_file_path)
            
            # Set initial conditions (10% of range)
            initial_conditions = [
                [bound[0] + (bound[1] - bound[0]) * 0.1 for bound in self.bounds]
            ]
            
            # Run optimization
            best_result = self.system_optimizer.optimize_system(self.bounds, initial_conditions)
            
            if best_result:
                # Print and return results
                print("\nBest configuration found:")
                for key, value in best_result.items():
                    print(f"{key}: {value:.3f}")
                
                return {
                    'Latitude': latitude,
                    'Longitude': longitude,
                    'Location ID': location_id,
                    **best_result
                }
            else:
                print("Optimization failed to converge")
                return {}
                
        except Exception as e:
            print(f"Error processing location: {str(e)}")
            return {}

def main():
    """Main execution."""
    # ======= Configuration =======
    # File paths
    yaml_file_path = "../input_yaml/input_file_chunk_0.yaml"  # Base configuration file
    csv_path = "../deposit_data/auCopper_chunk_0.csv"         # Location data
    output_path = "../simulation_results/simulation_results_chunk_0.csv"  # Results output
    
    # API credentials (replace with your own)
    api_key = "YOUR-NREL-API-KEY"  # Get from https://developer.nrel.gov/
    email = "your.email@example.com"  # Required for NERL API
    
    # Set NREL API key
    set_developer_nrel_gov_key(api_key)
    
    # ======= Initialize Optimizer =======
    optimizer = HybridOptimizer(
        yaml_file_path=yaml_file_path,
        api_key=api_key,
        email=email,
        enable_flexible_load=False,      # Set to True to enable flexible load
        max_load_reduction_percentage=0.2  # Only used if flexible load is enabled
    )
    
    # ======= Run Single Location Example =======
    # Example coordinates (replace with your location)
    test_location = {
        'latitude': -33.5265,
        'longitude': 149.1588,
        'id': "TEST_LOCATION"
    }
    
    result = optimizer.process_location(
        latitude=test_location['latitude'],
        longitude=test_location['longitude'],
        location_id=test_location['id']
    )
    
    if result:
        print("\nSingle location optimization successful!")
        print(f"LCOE: ${result['System LCOE ($/kWh)']:.4f}/kWh")
        print(f"Total System Cost: ${result['System Cost ($)']:,.2f}")
        print(f"Demand Met: {result['Demand Met Percentage']:.1f}%")
    
    # ======= Run Multiple Locations Example =======
    # Comment out if you only want to run single location
    """
    # Load location data
    locations = pd.read_csv(csv_path)
    
    # Process all locations
    results = []
    for _, row in locations.iterrows():
        result = optimizer.process_location(
            latitude=row['DEPOSIT_LATITUDE'],
            longitude=row['DEPOSIT_LONGITUDE'],
            location_id=row['DEPOSIT_UID']
        )
        if result:
            results.append(result)
    
    # Save and summarize results
    if results:
        results_df = pd.DataFrame(results)
        os.makedirs(os.path.dirname(output_path), exist_ok=True)
        results_df.to_csv(output_path, index=False)
        
        print("\nOptimization complete. Summary:")
        print(f"Total locations processed: {len(results)}")
        print(f"Average LCOE: ${results_df['System LCOE ($/kWh)'].mean():.4f}/kWh")
        print(f"Best LCOE: ${results_df['System LCOE ($/kWh)'].min():.4f}/kWh")
    """

if __name__ == "__main__":
    main()

/home/z5142067/miniconda3/envs/microgrid/lib/python3.8/site-packages/hopp/examples/parallel_simulations/scripts_modularised/log/hybrid_systems_2024-11-08T23.22.46.623487.log



Processing location AUS.NSW.333484 at (-33.5265, 149.1588)
Using existing solar data file: /home/z5142067/miniconda3/envs/microgrid/lib/python3.8/site-packages/hopp/simulation/resource_files/solar/-33.5265_149.1588_psmv3_60_2020.csv
Using existing wind data file: /home/z5142067/miniconda3/envs/microgrid/lib/python3.8/site-packages/hopp/simulation/resource_files/wind/-33.5265_149.1588_NASA_2020_60min_50m.srw



Best configuration found:
PV Capacity (kW): 29292.000
Wind Turbine Capacity (kW): 1000.000
Genset Capacity (kW): 17000.000
Battery Energy Capacity (kWh): 5000.000
Battery Power Capacity (kW): 2610.000
Total System Generation (kWh): 2554244484.780
Total PV Generation (kWh): 1343766814.903
Total Wind Generation (kWh): 9670029.576
Total Genset Generation (kWh): 1200807640.300
Total Battery Generation (kWh): -61724.192
Total CO2 emissions (tonne): 785928.601
System NPC ($): 142823213.986
System LCOE ($/kWh): 0.303
Deficit Hours Fixed: 7625.000
Total Load Reduction (kWh): 0.000
Total Load Served (kWh): 1969558247.650
Total Charging Prevented (kWh): 15706572.412
Demand Not Served (kWh): 0.000
Load Reduction Percentage: 0.000
Demand Met Percentage: 100.000
Project Lifetime (years): 25.000

Processing location AUS.NSW.333485 at (-31.1629, 145.6538)
Using existing solar data file: /home/z5142067/miniconda3/envs/microgrid/lib/python3.8/site-packages/hopp/simulation/resource_files/solar/-31.1629


Best configuration found:
PV Capacity (kW): 23616.000
Wind Turbine Capacity (kW): 9000.000
Genset Capacity (kW): 17000.000
Battery Energy Capacity (kWh): 5000.000
Battery Power Capacity (kW): 2373.000
Total System Generation (kWh): 2636724727.619
Total PV Generation (kWh): 1239359997.144
Total Wind Generation (kWh): 495333792.198
Total Genset Generation (kWh): 902030938.276
Total Battery Generation (kWh): -56863.015
Total CO2 emissions (tonne): 590379.249
System NPC ($): 121744047.945
System LCOE ($/kWh): 0.258
Deficit Hours Fixed: 7250.000
Total Load Reduction (kWh): 0.000
Total Load Served (kWh): 1969558247.650
Total Charging Prevented (kWh): 14158275.671
Demand Not Served (kWh): 0.000
Load Reduction Percentage: 0.000
Demand Met Percentage: 100.000
Project Lifetime (years): 25.000

Processing location AUS.NSW.334161 at (-31.5001, 145.8422)
Using existing solar data file: /home/z5142067/miniconda3/envs/microgrid/lib/python3.8/site-packages/hopp/simulation/resource_files/solar/-31.500


Best configuration found:
PV Capacity (kW): 22846.000
Wind Turbine Capacity (kW): 9000.000
Genset Capacity (kW): 17000.000
Battery Energy Capacity (kWh): 5000.000
Battery Power Capacity (kW): 2356.000
Total System Generation (kWh): 2592088308.780
Total PV Generation (kWh): 1190684776.579
Total Wind Generation (kWh): 500723170.559
Total Genset Generation (kWh): 900680361.642
Total Battery Generation (kWh): -57465.129
Total CO2 emissions (tonne): 589495.297
System NPC ($): 121380055.708
System LCOE ($/kWh): 0.257
Deficit Hours Fixed: 7225.000
Total Load Reduction (kWh): 0.000
Total Load Served (kWh): 1969558247.650
Total Charging Prevented (kWh): 14047029.131
Demand Not Served (kWh): 0.000
Load Reduction Percentage: 0.000
Demand Met Percentage: 100.000
Project Lifetime (years): 25.000

Results saved to ../simulation_results_modularised/combined_results_without_DR/lowest_cost_configs_chunk_0_PDR.csv

Optimization complete. Summary of results:
Total locations processed: 3
Average LCOE: 0.